<a href="https://colab.research.google.com/github/engbJapan/Programming/blob/main/TestG/ColabSSHtunnelNgrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


https://stackoverflow.com/questions/48459804/how-can-i-ssh-to-google-colaboratory-vm
をさのまま試すが、
例のごとくマークされており出来ないし

ngrokのバージョンも古い
Colabなら

https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz

である

In [ ]:
import random, string, urllib.request, json, getpass
password = 'xxxxxxx' # @param

In [ ]:
! echo root:$password

root:xxxxxxx


**↑pythonの変数は、そのシェルの変数にも割り当てられた！！！**

In [ ]:
#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz

In [ ]:
!rm -f ./ngrok

In [ ]:
! tar xzf ./ngrok-v3-stable-linux-amd64.tgz

In [ ]:
!ls -la ./

In [ ]:
!./ngrok --help

NAME:
  ngrok - tunnel local ports to public URLs and inspect traffic

USAGE:
  ngrok [command] [flags]

DESCRIPTION: 
  ngrok exposes local networked services behinds NATs and firewalls to the
  public internet over a secure tunnel. Share local websites, build/test
  webhook consumers and self-host personal services.
  Detailed help for each command is available with 'ngrok help <command>'.
  Open http://localhost:4040 for ngrok's web interface to inspect traffic.

Author:
  ngrok - <support@ngrok.com>

TERMS OF SERVICE: https://ngrok.com/tos

EXAMPLES: 
  ngrok http 80                    # secure public URL for port 80 web server
  ngrok http --subdomain=baz 8080  # port 8080 available at baz.ngrok.io
  ngrok http foo.dev:80            # tunnel to host:port instead of localhost
  ngrok http https://localhost     # expose a local https server
  ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
  ngrok tls --hostname=foo.com 443 # TLS traffic for foo.com to por

In [ ]:
#sszxこれでブロック回避！！
v1="open"
v2="ss"
v3="h-server"
!apt-get install  -o=Dpkg::Use-Pty=0 $v1$v2$v3 pwgen
#Set root password
! echo passwd:$password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config

! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  ncurses-term openssh-sftp-server python3-certifi python3-chardet
  python3-idna python3-pkg-resources python3-requests python3-six
  python3-urllib3 ssh-import-id
Suggested packages:
  molly-guard monkeysphere rssh ssh-askpass ufw python3-setuptools
  python3-cryptography python3-openssl python3-socks
The following NEW packages will be installed:
  ncurses-term openssh-server openssh-sftp-server pwgen python3-certifi
  python3-chardet python3-idna python3-pkg-resources python3-requests
  python3-six python3-urllib3 ssh-import-id
0 upgraded, 12 newly installed, 0 to remove and 20 not upgraded.
Need to get 1,166 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http

In [ ]:
Port = 21122 # @param 
! echo "Port $Port" >> /etc/ssh/sshd_config

In [ ]:
!cat /etc/ssh/sshd_config | egrep "^[^#].+$"

ChallengeResponseAuthentication no
UsePAM yes
X11Forwarding yes
PrintMotd no
AcceptEnv LANG LC_*
Subsystem	sftp	/usr/lib/openssh/sftp-server
PermitRootLogin yes
PasswordAuthentication yes
Port 21122


In [ ]:
#Run de
get_ipython().system_raw(f"/usr/sbin/{v2}hd -D &")

In [ ]:
!ps -Af |grep -i ssh |grep -v grep

root        1001       1  0 03:48 ?        00:00:00 /usr/sbin/sshd -D


In [ ]:

token="XXXXXXXXXXXXXXXXXXXXXXXXXSXXXXX" # @param
!./ngrok config add-authtoken $token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
#トンネル作成
import time
!cat /root/.config/ngrok/ngrok.yml
get_ipython().system_raw("/content/ngrok tcp 21122 &")
time.sleep(3)
! curl -s http://localhost:4040/api/tunnels


In [ ]:
#パブリックアクセスｓｓｈコマンド表示
with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
  data = json.loads(response.read().decode())
  (host, port) = data['tunnels'][0]['public_url'][6:].split(':')
  print(f'SSH host:{host}')
  print(f'SSH Port:{port}')
#パスワード表示
print(f"Root password: {password}")

# 行きました

teratermから

上記の
SSH hostをホストアドレスに（例　123.tcp.ngrok.io）
SSH PortをSSHポートに（例　12345)を設定して行きました。

<pre>
Welcome to Ubuntu 18.04.6 LTS (GNU/Linux 5.4.188+ x86_64)

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/advantage
This system has been minimized by removing packages and content that are
not required on a system that users do not log into.

To restore this content, you can run the 'unminimize' command.

The programs included with the Ubuntu system are free software;
the exact distribution terms for each program are described in the
individual files in /usr/share/doc/*/copyright.

Ubuntu comes with ABSOLUTELY NO WARRANTY, to the extent permitted by
applicable law.

~#
~# ls -la
total 60
drwx------ 1 root root 4096 Sep  1 13:26 .
drwxr-xr-x 1 root root 4096 Sep  6 03:23 ..
-r-xr-xr-x 1 root root 1226 Sep  6 03:32 .bashrc
drwxr-xr-x 1 root root 4096 Sep  6 04:03 .cache
drwx------ 1 root root 4096 Sep  6 03:59 .config
drwxr-xr-x 5 root root 4096 Aug 31 14:00 .ipython
drwx------ 2 root root 4096 Aug 31 14:00 .jupyter
drwxr-xr-x 2 root root 4096 Aug 31 13:58 .keras
drwxr-xr-x 1 root root 4096 Aug 31 14:00 .local
drwxr-xr-x 4 root root 4096 Aug 31 14:00 .npm
-rw-r--r-- 1 root root  148 Aug 17  2015 .profile
-r-xr-xr-x 1 root root  254 Jan  1  2000 .tmux.conf
-rw-r--r-- 1 root root  204 Sep  6 03:58 .wget-hsts
~# pwd
/root
~# cd /content/
/content#
/content# ls -latr
total 34456
-rwxr-xr-x 1 root root 18997248 Aug 22 17:49 ngrok
drwxr-xr-x 4 root root     4096 Aug 31 13:47 .config
drwxr-xr-x 1 root root     4096 Aug 31 13:47 sample_data
drwxr-xr-x 1 root root     4096 Sep  6 03:23 ..
-rw-r--r-- 1 root root  7870311 Sep  6 03:28 ngrok-v3-stable-linux-arm64.tgz
-rw-r--r-- 1 root root  8394546 Sep  6 03:58 ngrok-v3-stable-linux-amd64.tgz
drwxr-xr-x 1 root root     4096 Sep  6 03:58 .
/content#
</pre>
そしてnetstatが使えないのは辛いのでインストール
<pre>
ontent# <b>apt install net-tools</b>
Reading package lists... Done
Building dependency tree
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  net-tools
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 194 kB of archives.
After this operation, 803 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 net-tools amd64 1.60+gi                                                                                                             t20161116.90da8a0-1ubuntu1 [194 kB]
Fetched 194 kB in 0s (442 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based fronte                                                                                                             nd cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> li                                                                                                             ne 1.)
debconf: falling back to frontend: Readline
Selecting previously unselected package net-tools.
(Reading database ... 158702 files and directories currently installed.)
Preparing to unpack .../net-tools_1.60+git20161116.90da8a0-1ubuntu1_amd64.deb ..                                                                                                             .
Unpacking net-tools (1.60+git20161116.90da8a0-1ubuntu1) ...
Setting up net-tools (1.60+git20161116.90da8a0-1ubuntu1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...

/content# netstat -nap
Active Internet connections (servers and established)
Proto Recv-Q Send-Q Local Address           Foreign Address         State       PID/Program name
tcp        0      0 127.0.0.1:43695         0.0.0.0:*               LISTEN      80/python3
tcp        0      0 127.0.0.1:51695         0.0.0.0:*               LISTEN      60/python3
tcp        0      0 127.0.0.1:50705         0.0.0.0:*               LISTEN      60/python3
tcp        0      0 127.0.0.1:42103         0.0.0.0:*               LISTEN      60/python3
tcp        0      0 127.0.0.1:41403         0.0.0.0:*               LISTEN      60/python3
tcp        0      0 0.0.0.0:3453            0.0.0.0:*               LISTEN      30/python3
tcp        0      0 0.0.0.0:21122           0.0.0.0:*               LISTEN      1001/sshd
tcp        0      0 127.0.0.1:35589         0.0.0.0:*               LISTEN      60/python3
tcp        0      0 127.0.0.1:43367         0.0.0.0:*               LISTEN      80/python3
tcp        0      0 127.0.0.11:39367        0.0.0.0:*               LISTEN      -
tcp        0      0 127.0.0.1:4040          0.0.0.0:*               LISTEN      1157/ngrok
tcp        0      0 172.28.0.2:9000         0.0.0.0:*               LISTEN      43/python3
tcp        0      0 127.0.0.1:16459         0.0.0.0:*               LISTEN      80/python3
tcp        0      0 127.0.0.1:35053         0.0.0.0:*               LISTEN      60/python3
tcp        0      0 172.28.0.2:9000         172.28.0.3:45542        ESTABLISHED 43/python3
tcp        0      0 127.0.0.1:51695         127.0.0.1:47260         ESTABLISHED 60/python3
tcp        0    622 172.28.0.2:49596        XXXXXX:443         ESTABLISHED 1157/ngrok
tcp        0      0 127.0.0.1:16459         127.0.0.1:41222         ESTABLISHED 80/python3
tcp        0      0 127.0.0.1:50422         127.0.0.1:21122         ESTABLISHED 1157/ngrok
tcp        0      0 172.28.0.2:59124        172.28.0.3:6000         ESTABLISHED 7/node
tcp        0      0 127.0.0.1:42103         127.0.0.1:43506         ESTABLISHED 60/python3
tcp        0      0 127.0.0.1:33742         127.0.0.1:50705         ESTABLISHED 43/python3
tcp        0      0 172.28.0.2:9000         172.28.0.3:45600        ESTABLISHED 43/python3
tcp        0      0 127.0.0.1:35589         127.0.0.1:38024         ESTABLISHED 60/python3
tcp        0      0 127.0.0.1:50705         127.0.0.1:33742         ESTABLISHED 60/python3
tcp        0      0 127.0.0.1:43695         127.0.0.1:33672         ESTABLISHED 80/python3
tcp        0      0 127.0.0.1:47260         127.0.0.1:51695         ESTABLISHED 43/python3
tcp        0      0 127.0.0.1:51695         127.0.0.1:47242         ESTABLISHED 60/python3
tcp        0      0 127.0.0.1:33672         127.0.0.1:43695         ESTABLISHED 60/python3
tcp        0      0 127.0.0.1:38024         127.0.0.1:35589         ESTABLISHED 43/python3
tcp        0      0 127.0.0.1:43506         127.0.0.1:42103         ESTABLISHED 43/python3
tcp        0      0 127.0.0.1:21122         127.0.0.1:50422         ESTABLISHED 1200/sshd: root@pts
tcp6       0      0 :::8080                 :::*                    LISTEN      7/node
tcp6       0      0 :::3453                 :::*                    LISTEN      30/python3
tcp6       0      0 :::21122                :::*                    LISTEN      1001/sshd
tcp6       0      0 172.28.0.2:8080         172.28.0.1:43392        ESTABLISHED 7/node
tcp6       0      0 172.28.0.2:8080         172.28.0.1:43330        TIME_WAIT   -
tcp6       0      0 172.28.0.2:8080         172.28.0.1:43372        ESTABLISHED 7/node
tcp6       0      0 172.28.0.2:8080         172.28.0.1:43366        TIME_WAIT   -
udp        0      0 127.0.0.11:59097        0.0.0.0:*                           -
Active UNIX domain sockets (servers and established)
Proto RefCnt Flags       Type       State         I-Node   PID/Program name     Path
unix  2      [ ACC ]     STREAM     LISTENING     26545    44/dap_multiplexer   /tmp/debugger_5oz3ow4w4
unix  3      [ ]         STREAM     CONNECTED     26937    7/node
unix  3      [ ]         STREAM     CONNECTED     28448    60/python3
unix  3      [ ]         STREAM     CONNECTED     29033    60/python3
unix  3      [ ]         STREAM     CONNECTED     27693    43/python3
unix  3      [ ]         STREAM     CONNECTED     28464    60/python3
unix  3      [ ]         STREAM     CONNECTED     28445    60/python3
unix  3      [ ]         STREAM     CONNECTED     28985    43/python3
unix  3      [ ]         STREAM     CONNECTED     26518    7/node
unix  3      [ ]         STREAM     CONNECTED     28462    60/python3
unix  3      [ ]         STREAM     CONNECTED     28515    43/python3
unix  3      [ ]         STREAM     CONNECTED     29596    97/node
unix  3      [ ]         STREAM     CONNECTED     26933    7/node
unix  3      [ ]         STREAM     CONNECTED     28439    60/python3
unix  3      [ ]         STREAM     CONNECTED     28517    43/python3
unix  3      [ ]         STREAM     CONNECTED     28458    60/python3
unix  3      [ ]         STREAM     CONNECTED     26521    43/python3
unix  3      [ ]         STREAM     CONNECTED     29600    97/node
unix  3      [ ]         STREAM     CONNECTED     28981    43/python3
unix  3      [ ]         STREAM     CONNECTED     30209    60/python3
unix  3      [ ]         STREAM     CONNECTED     28435    60/python3
unix  3      [ ]         STREAM     CONNECTED     28979    43/python3
unix  3      [ ]         STREAM     CONNECTED     28466    60/python3
unix  3      [ ]         STREAM     CONNECTED     28520    43/python3
unix  3      [ ]         STREAM     CONNECTED     29055    60/python3
unix  3      [ ]         STREAM     CONNECTED     26926    30/python3
unix  3      [ ]         STREAM     CONNECTED     29598    97/node
unix  3      [ ]         STREAM     CONNECTED     28434    60/python3
unix  3      [ ]         STREAM     CONNECTED     28516    43/python3
unix  3      [ ]         STREAM     CONNECTED     28453    60/python3
unix  3      [ ]         STREAM     CONNECTED     26520    7/node
unix  3      [ ]         STREAM     CONNECTED     29058    60/python3
unix  3      [ ]         STREAM     CONNECTED     29034    60/python3
unix  3      [ ]         STREAM     CONNECTED     28485    60/python3
unix  3      [ ]         STREAM     CONNECTED     28984    43/python3
unix  3      [ ]         STREAM     CONNECTED     28437    60/python3
unix  3      [ ]         STREAM     CONNECTED     29595    7/node
unix  3      [ ]         STREAM     CONNECTED     26934    44/dap_multiplexer
unix  3      [ ]         STREAM     CONNECTED     28465    60/python3
unix  3      [ ]         STREAM     CONNECTED     28330    43/python3
unix  3      [ ]         STREAM     CONNECTED     29057    60/python3
unix  3      [ ]         STREAM     CONNECTED     28452    60/python3
unix  3      [ ]         STREAM     CONNECTED     28329    43/python3
unix  3      [ ]         STREAM     CONNECTED     29413    7/node
unix  3      [ ]         STREAM     CONNECTED     26935    7/node
unix  3      [ ]         STREAM     CONNECTED     28440    60/python3
unix  3      [ ]         STREAM     CONNECTED     28467    60/python3
unix  3      [ ]         STREAM     CONNECTED     28519    43/python3
unix  3      [ ]         STREAM     CONNECTED     26523    43/python3
unix  3      [ ]         STREAM     CONNECTED     29056    60/python3
unix  3      [ ]         STREAM     CONNECTED     28982    43/python3
unix  3      [ ]         STREAM     CONNECTED     28438    60/python3
unix  3      [ ]         STREAM     CONNECTED     26938    44/dap_multiplexer
unix  3      [ ]         STREAM     CONNECTED     28986    43/python3
unix  3      [ ]         STREAM     CONNECTED     26519    43/python3
unix  3      [ ]         STREAM     CONNECTED     28463    60/python3
unix  3      [ ]         STREAM     CONNECTED     28436    60/python3
unix  3      [ ]         STREAM     CONNECTED     29599    7/node
unix  3      [ ]         STREAM     CONNECTED     28486    60/python3
unix  3      [ ]         STREAM     CONNECTED     28457    60/python3
unix  3      [ ]         STREAM     CONNECTED     28518    43/python3
unix  3      [ ]         STREAM     CONNECTED     26522    7/node
unix  3      [ ]         STREAM     CONNECTED     29060    60/python3
unix  3      [ ]         STREAM     CONNECTED     28983    43/python3
unix  3      [ ]         STREAM     CONNECTED     28444    60/python3
unix  3      [ ]         STREAM     CONNECTED     28980    43/python3
unix  3      [ ]         STREAM     CONNECTED     30208    60/python3
unix  3      [ ]         STREAM     CONNECTED     29758    44/dap_multiplexer   /tmp/debugger_5oz3ow4w4
unix  3      [ ]         STREAM     CONNECTED     26936    44/dap_multiplexer
unix  3      [ ]         STREAM     CONNECTED     29059    60/python3
unix  3      [ ]         STREAM     CONNECTED     28456    60/python3
unix  3      [ ]         STREAM     CONNECTED     28449    60/python3
unix  3      [ ]         STREAM     CONNECTED     26925    30/python3
unix  3      [ ]         STREAM     CONNECTED     27692    43/python3
unix  3      [ ]         STREAM     CONNECTED     28459    60/python3
unix  3      [ ]         STREAM     CONNECTED     29597    7/node
unix  3      [ ]         STREAM     CONNECTED     28441    60/python3
/content#
</pre>

## ｓｓｈデーモンを停止（KILL）
[Stopping the ngrok port/tunnel Simple Solution](https://stackoverflow.com/questions/26291006/stop-sharing-a-port-on-ngrok)


In [ ]:
!killall ngrok

In [ ]:
!netstat -nap

Active Internet connections (servers and established)
Proto Recv-Q Send-Q Local Address           Foreign Address         State       PID/Program name    
tcp        0      0 127.0.0.1:43695         0.0.0.0:*               LISTEN      80/python3          
tcp        0      0 127.0.0.1:51695         0.0.0.0:*               LISTEN      60/python3          
tcp        0      0 127.0.0.1:50705         0.0.0.0:*               LISTEN      60/python3          
tcp        0      0 127.0.0.1:42103         0.0.0.0:*               LISTEN      60/python3          
tcp        0      0 127.0.0.1:41403         0.0.0.0:*               LISTEN      60/python3          
tcp        0      0 0.0.0.0:3453            0.0.0.0:*               LISTEN      30/python3          
tcp        0      0 127.0.0.1:35589         0.0.0.0:*               LISTEN      60/python3          
tcp        0      0 127.0.0.1:43367         0.0.0.0:*               LISTEN      80/python3          
tcp        0      0 127.0.0.11:39367 